In [1]:
import matplotlib.pyplot as plt
import pandas as pd
from PIL import Image
import numpy as np

In [2]:
label=pd.read_csv('TrainDev/train.csv',header=None)
label=label.dropna(axis=1,how='all')
label.shape

/usr/lib/python3/dist-packages/IPython/core/interactiveshell.py:3062: DtypeWarning: Columns (30,35,40,45,50,55,60,65,70,75,80,85,90,95,100,105,110,115,120,125,130,135) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


(25768, 136)

In [3]:
data=label[[0,1,2,3,4,5]]
for i in range(1,int((label.shape[1]-1)/5)):
    current=5*i
    add_index=[0,current+1,current+2,current+3,current+4,current+5]
    add_index=label[add_index]
    add_index.columns=[0,1,2,3,4,5]
    data=data.append(add_index,ignore_index=True)
data=data.dropna(axis=0)
data.columns=['file','x1','y1','x2','y2','damage']
data['x2']+=data['x1']
data['y2']+=data['y1']
data.index=[i for i in range(data.shape[0])]
data

,file,x1,y1,x2,y2,damage
0,38414.jpg,46.0,146.0,622.0,720.0,不良-機械傷害
1,03182.jpg,581.0,277.0,678.0,370.0,不良-機械傷害
2,29863.jpg,514.0,538.0,631.0,682.0,不良-機械傷害
3,17937.jpg,658.0,263.0,717.0,337.0,不良-機械傷害
4,40878.jpg,432.0,160.0,548.0,292.0,不良-機械傷害
...,...,...,...,...,...,...
43365,02176.jpg,838.0,636.0,868.0,657.0,不良-炭疽病
43366,48869.jpg,181.0,435.0,229.0,473.0,不良-炭疽病
43367,02176.jpg,547.0,240.0,567.0,258.0,不良-炭疽病
43368,02176.jpg,514.0,257.0,530.0,271.0,不良-炭疽病


In [4]:
x_train=[]
for i in range(int(data.shape[0]/2)):
    x=Image.open('TrainDev/Pics/'+data['file'][i])
    x=x.crop((data['x1'][i],data['y1'][i],data['x2'][i],data['y2'][i]))
    x=x.resize((200,200))
    x=np.array(x)
    x_train.append(x)
x_train=np.array(x_train)
y_train=data['damage']
y_train=y_train[:int(y_train.shape[0]/2)]
y_train=pd.get_dummies(y_train)
y_train=y_train.to_numpy()
print('y_train shape: ',y_train.shape)
print('x_train shape: ',x_train.shape)

y_train shape:  (21685, 5)
x_train shape:  (21685, 200, 200, 3)


## Training

In [5]:
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.compat.v1 import ConfigProto, InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth = True
config.gpu_options.per_process_gpu_memory_fraction = 0.6
sess = InteractiveSession(config=config)

In [6]:
cnn=models.Sequential()
#Layer1
cnn.add(layers.Input((200,200,3)))
cnn.add(layers.Conv2D(32,3,activation='relu'))
cnn.add(layers.Conv2D(32,3,activation='relu'))
cnn.add(layers.BatchNormalization())
cnn.add(layers.MaxPool2D(2))

#Layer2
cnn.add(layers.Conv2D(64,3,activation='relu'))
cnn.add(layers.Conv2D(64,3,activation='relu'))
cnn.add(layers.BatchNormalization())
cnn.add(layers.MaxPool2D(2))

#Layer3
cnn.add(layers.Conv2D(128,3,activation='relu'))
cnn.add(layers.Conv2D(128,3,activation='relu'))
cnn.add(layers.BatchNormalization())
cnn.add(layers.MaxPool2D(2))

cnn.add(layers.Flatten())
cnn.add(layers.Dropout(0.4))
cnn.add(layers.Dense(1024,activation='relu'))
cnn.add(layers.Dense(512,activation='relu'))
cnn.add(layers.Dense(5,activation='softmax'))
cnn.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 198, 198, 32)      896       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 196, 196, 32)      9248      
_________________________________________________________________
batch_normalization (BatchNo (None, 196, 196, 32)      128       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 98, 98, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 96, 96, 64)        18496     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 94, 94, 64)        36928     
_________________________________________________________________
batch_normalization_1 (Batch (None, 94, 94, 64)        2

In [7]:
cnn.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
cnn.fit(x_train,y_train,epochs=30)

Epoch 1/50
678/678 [==============================] - 61s 90ms/step - loss: 1.1613 - accuracy: 0.7742
Epoch 2/50
678/678 [==============================] - 61s 90ms/step - loss: 0.5359 - accuracy: 0.8306
Epoch 3/50
678/678 [==============================] - 61s 91ms/step - loss: 0.5151 - accuracy: 0.8398
Epoch 4/50
678/678 [==============================] - 62s 91ms/step - loss: 0.4699 - accuracy: 0.8537
Epoch 5/50
678/678 [==============================] - 62s 91ms/step - loss: 0.4300 - accuracy: 0.8670
Epoch 6/50
678/678 [==============================] - 62s 91ms/step - loss: 0.3881 - accuracy: 0.8790
Epoch 7/50
678/678 [==============================] - 62s 91ms/step - loss: 0.3623 - accuracy: 0.8853
Epoch 8/50
678/678 [==============================] - 62s 91ms/step - loss: 0.3194 - accuracy: 0.8972
Epoch 9/50
678/678 [==============================] - 62s 92ms/step - loss: 0.2820 - accuracy: 0.9106
Epoch 10/50
678/678 [==============================] - 62s 92ms/step - loss: 0.227

In [8]:
cnn.save('mango_cnn.h5')